In [55]:
from pyautogui import press, typewrite, hotkey, keyDown
import time
time.sleep(0.3)
press('enter')
press('enter')
press('enter')
# hotkey('command', 'space')
keyDown('a')
keyDown('a')
keyDown('a')
keyDown('a')
# typewrite('a')
# typewrite('quick brown fox')
# hotkey('command', 'space')

typewrite('Hello World!!!', interval=0.2)
keyDown('a')
hotkey('tab')
# time.sleep(0.3)
# typewrite('idot')
# hotkey('enter')

In [ ]:
aquick brown fox
aäe
aHello World!!!aquickaaaaaaaaa

aaaaHello World111a

In [ ]:
def execute(inst):
    
    

In [1]:
import pygame, sys
import pygame.locals

pygame.init()
BLACK = (0,0,0)
WIDTH = 1280
HEIGHT = 1024
windowSurface = pygame.display.set_mode((WIDTH, HEIGHT), 0, 32)

windowSurface.fill(BLACK)

while True:
    for event in pygame.event.get():
        if event.key == pygame.K_p:
             #Do what you want to here
            pass
        if event.type == pygame.locals.QUIT:
            pygame.quit()
            sys.exit()

ModuleNotFoundError: No module named 'pygame'

In [4]:
import sys, os
import termios, fcntl
import select

fd = sys.stdin.fileno()
newattr = termios.tcgetattr(fd)
newattr[3] = newattr[3] & ~termios.ICANON
newattr[3] = newattr[3] & ~termios.ECHO
termios.tcsetattr(fd, termios.TCSANOW, newattr)

oldterm = termios.tcgetattr(fd)
oldflags = fcntl.fcntl(fd, fcntl.F_GETFL)
fcntl.fcntl(fd, fcntl.F_SETFL, oldflags | os.O_NONBLOCK)

print ("Type some stuff")
while True:
    inp, outp, err = select.select([sys.stdin], [], [])
    c = sys.stdin.read()
    if c == 'q':
        break
    print ("-", c)

# Reset the terminal:
termios.tcsetattr(fd, termios.TCSAFLUSH, oldterm)
fcntl.fcntl(fd, fcntl.F_SETFL, oldflags)

error: (25, 'Inappropriate ioctl for device')

In [7]:
import sys,tty
tty.setcbreak(sys.stdin)  
key = ord(sys.stdin.read(1))  # key captures the key-code 

if key==97:
    print ("you pressed a")
else:
    print ("you pressed something else ..."  )
sys.exit(0)

error: (25, 'Inappropriate ioctl for device')

In [11]:
def capture(device="/dev/spidev0.1"):
    with Lepton(device) as l:
        data, _ = l.capture()
        print(data)

In [12]:
capture()

NameError: name 'Lepton' is not defined

In [15]:
from PIL import ImageGrab
import time

# time.sleep(5)
ImageGrab.grab().save("screen_capture.jpg", "PNG")

In [34]:
import numpy as np 
import cv2 
# for windows, mac users
# from PIL import ImageGrab
# for linux users
from PIL import ImageGrab
import time

# four character code object for video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# video writer object
out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1366, 768))

while True:
    start = time.time()
    # capture computer screen
    img = ImageGrab.grab()
    # convert image to numpy array
    img_np = np.array(img)
    # convert color space from BGR to RGB
    frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (680,480))
    # show image on OpenCV frame
#     cv2.imshow("Screen", frame)
    # write frame to video writer
#     out.write(frame)
    print("FPS:",1/(time.time() - start))
    if cv2.waitKey(1) == 27:
        break

out.release()
cv2.destroyAllWindows()

FPS: 2.824205606658555
FPS: 3.2862968845123524
FPS: 3.509537149531928
FPS: 3.7473835437993963
FPS: 3.5421065476929416
FPS: 3.5645027097265456
FPS: 3.696503294355706
FPS: 4.00888504872631
FPS: 3.537684915827926
FPS: 3.5419569878489447
FPS: 3.7001392965304665
FPS: 3.644713940237628
FPS: 3.429425510656286
FPS: 3.6397767348865626
FPS: 1.868554033783895
FPS: 1.5346296611363266
FPS: 1.274958606365151
FPS: 3.68453074831665
FPS: 3.6528335276899373
FPS: 3.5950643019018003
FPS: 3.6523532198289073


KeyboardInterrupt: 

In [35]:
import pyautogui
import cv2
import time
while True: 
    start = time.time()
    pic = pyautogui.screenshot()
    cv2.imshow("Screen", np.array(pic))
    print("FPS:",1/(time.time() - start))
    if cv2.waitKey(1) == 27:
        break

out.release()
cv2.destroyAllWindows()
    

FPS: 2.6324667450784474
FPS: 3.379164937863804
FPS: 3.2330153738240317
FPS: 3.817494397955413
FPS: 3.395688904900885
FPS: 3.409545873927481
FPS: 2.4441462162475576
FPS: 1.87109308258756
FPS: 1.897687920377085
FPS: 3.1675014046556025
FPS: 3.591671761778271
FPS: 2.4760494701732636
FPS: 1.8558037892996817


In [7]:
bytearray.fromhex("C8").decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 0: unexpected end of data